# TIC TAC TOE draft

In [26]:
import numpy as np
import math
import itertools
import numpy as np

In [27]:
grid = np.zeros((3,3),dtype='uint8')

In [28]:
p1_pawns = np.array([v for v in range(1,10,2)])
p2_pawns = np.array([v for v in range(2,10,2)])

In [29]:
def get_grid(values, size=(3,3), cell_height=3, cell_width=3, horizontal_sep = "-", vertical_sep = "|", cross_sep = "+"):
    
    def get_cell(empty=False):        
        value = "{}"
        
        if empty:
            value = "  "
            
        margin_left, margin_right = round((cell_width-2) / 2), math.ceil((cell_width-2) / 2)        
        return " " * margin_left + value + " " * margin_right 
        
    def get_horizontal_row():                
        margin_top, margin_bottom = round((cell_height-1) / 2), math.ceil((cell_height-1) / 2)
        
        return ((vertical_sep.join([get_cell(empty=True) for i in range(size[1])]) + "\n") * margin_top + \
            vertical_sep.join([get_cell() for i in range(size[1])]) + "\n" + \
            (vertical_sep.join([get_cell(empty=True) for i in range(size[1])]) + "\n") * margin_bottom)[:-1]
    
    def get_horizontal_line():                
        return cross_sep.join([horizontal_sep * cell_width for i in range(size[1])])
    
    def get_empty_grid():
        return ("\n"+ get_horizontal_line() + "\n").join([get_horizontal_row() for i in range(size[0])])
    
    def fill_grid(grid, values):
        return grid.format(*list(map(lambda v: (" " + str(v))[-2:], values)))
    
    print(fill_grid(get_empty_grid(), values))
    
    
get_grid(np.zeros(9, dtype='uint8'), cell_height=3, cell_width=4)

    |    |    
  0 |  0 |  0 
    |    |    
----+----+----
    |    |    
  0 |  0 |  0 
    |    |    
----+----+----
    |    |    
  0 |  0 |  0 
    |    |    


## Game mechanisms

In [30]:
p1_pawns, p2_pawns

(array([1, 3, 5, 7, 9]), array([2, 4, 6, 8]))

In [31]:
game = np.concatenate((p1_pawns, p2_pawns), axis=0)
np.random.shuffle(game)
get_grid(game, cell_height=3, cell_width=4)


def check_state(values):
    grid = values.reshape(3,3)
    rows_sums = np.sum(grid, axis=1)
    cols_sums = np.sum(grid, axis=0)
    diag1_sum = np.array([grid[i,i] for i in range(0,grid.shape[0])]).sum()
    diag2_sum = np.array([np.fliplr(grid)[i,i] for i in range(0,grid.shape[0])]).sum()
    if 15 in np.concatenate((rows_sums, cols_sums, [diag1_sum], [diag2_sum]), axis=0):
        print("WIN ! ")
    return rows_sums, cols_sums, diag1_sum, diag2_sum
    
check_state(game)

    |    |    
  3 |  2 |  8 
    |    |    
----+----+----
    |    |    
  4 |  5 |  6 
    |    |    
----+----+----
    |    |    
  7 |  1 |  9 
    |    |    
WIN ! 


(array([13, 15, 17]), array([14,  8, 23]), 17, 20)

## Build all the possibilities of the grid

In [56]:
def get_combinations(arr):
    """
    Get all the combinations and permutations of the given array
    """
    # get all combinations
    all_combinations = [("".join(map(str, v)) + "0"*len(arr))[:len(arr)] for l in range(0,len(arr)+1) for v in list(itertools.combinations(arr, l))]   
    
    # get all permutations
    all_permutations = np.unique(["".join(permutation) for combination in all_combinations for permutation in itertools.permutations(combination)])
    
    return all_permutations

In [ ]:
arr = [1,2,3,4,5,6,7,8,9]
combinations = get_combinations(arr)
print(len(combinations))

## Check all the victorious grid

In [33]:
def get_victorious(grids, score=4):    
    grids = grids.reshape(-1,int(grids.shape[1]/2),int(grids.shape[1]/2))
    diags_scores = np.array([np.diag(g).sum() for g in grids])
    diags_scores_inverse = np.array([np.diag(np.fliplr(g)).sum() for g in grids])
        
    # rows wins
    wins_by_rows = np.where(np.sum(grids, axis=2) == score)    
    wins_by_rows = np.array(list(zip(wins_by_rows[0],wins_by_rows[1])))
    
    # columns wins
    wins_by_cols = np.where(np.sum(grids, axis=1) == score)
    wins_by_cols = np.array(list(zip(wins_by_cols[0],wins_by_cols[1])))
    
    # diagonals wins
    wins_by_diags = np.where(diags_scores == score)
    wins_by_diags_inverse = np.where(diags_scores_inverse == score)
    
    win_lines = np.unique(np.append(
        [k for k,j in wins_by_cols if not 0 in grids[k,:, j]],
        [k for k,j in wins_by_rows if not 0 in grids[k,j,:]]))
    
    win_lines = np.unique(np.append(
        win_lines,
        [k for k in wins_by_diags[0] if not 0 in np.diag(grids[k])]))
    
    win_lines = np.unique(np.append(
        win_lines,
        [k for k in wins_by_diags_inverse[0] if not 0 in np.diag(np.fliplr(grids[k]))]))
    
    if len(win_lines) > 0:
        return grids[win_lines]
    else:
        return []


## Build the tree

In [34]:
game_grids = get_combinations([1,2,3,4])
# victorious_grids = get_victorious(game_grids, score=15)

print("{} game grids".format(len(game_grids)))
#print("{} victorious grids".format(len(victorious_grids)))

step 1
[(), (1,), (2,), (3,), (4,), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4), (1, 2, 3), (1, 2, 4), (1, 3, 4), (2, 3, 4), (1, 2, 3, 4)]
step 2
208 game grids
